In [0]:
import pandas as pd

# Read data from review files 

In [0]:
reviewDataPath = {'yelp': 'yelp_labelled.txt',
                 'amazon': 'amazon_cells_labelled.txt',
                 'imdb': 'imdb_labelled.txt'}
reviewList = []

for source, filepath in reviewDataPath.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    # Add another column filled with the source name
    df['source'] = source 
    reviewList.append(df)

df = pd.concat(reviewList)

In [0]:
review_imdb = df[df['source'] == 'amazon']

In [7]:
review_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1000 non-null   object
 1   label     1000 non-null   int64 
 2   source    1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


In [8]:
# Just a look at data
print(df.iloc[:10])

                                            sentence  label source
0                           Wow... Loved this place.      1   yelp
1                                 Crust is not good.      0   yelp
2          Not tasty and the texture was just nasty.      0   yelp
3  Stopped by during the late May bank holiday of...      1   yelp
4  The selection on the menu was great and so wer...      1   yelp
5     Now I am getting angry and I want my damn pho.      0   yelp
6              Honeslty it didn't taste THAT fresh.)      0   yelp
7  The potatoes were like rubber and you could te...      0   yelp
8                          The fries were great too.      1   yelp
9                                     A great touch.      1   yelp


# Split data in train and test

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
review_yelp = df[df['source'] == 'yelp']

sentences = review_yelp['sentence'].values

y = review_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.25, random_state=1000)

In [11]:
sentences_train.size

750

# Tokenize data

In [12]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)


In [0]:
# Adding 1 because of reserved 0 index
# The indexing is ordered after the most common words in the text, 
# which you can see by the word the having the index 1. 
# It is important to note that the index 0 is reserved 
# and is not assigned to any word. This zero index is used for padding,
# because every statement is not of same size

vocab_size = len(tokenizer.word_index) + 1 

In [15]:
print(vocab_size)

1747


In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [17]:
print(sentences_train[1:6])

['Sorry, I will not be getting food from here anytime soon :('
 'Of all the dishes, the salmon was the best, but all were great.'
 'The fries were not hot, and neither was my burger.'
 "In fact I'm going to round up to 4 stars, just because she was so awesome."
 'Will go back next trip out.']


In [18]:
print(X_train[1])
print(X_train[2])
print(X_train[3])
print(X_train[4])

[740, 4, 46, 12, 20, 160, 10, 72, 35, 355, 232]
[11, 43, 1, 171, 1, 283, 3, 1, 47, 26, 43, 24, 22]
[1, 233, 24, 12, 209, 2, 741, 3, 23, 125]
[14, 356, 83, 126, 5, 742, 59, 5, 357, 96, 41, 127, 234, 3, 25, 161]


With Tokenizer, the resulting vectors equal the length of each text, and the numbers don’t denote counts,
but rather correspond to the word values from the dictionary tokenizer.word_index.

# PAD Sequance

Each text sequence has in most cases different length of words. 
To counter this, pad_sequence() is used ,which simply pads the sequence of words with zeros. 
By default, it prepends zeros but we want to append them.
Typically it does not matter whether you prepend or append zeros.

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
# maxlen parameter to specify how long the sequences should be. 
#This cuts sequences that exceed that number.

maxlen = 100

In [0]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [22]:
print(X_train[1])

[740   4  46  12  20 160  10  72  35 355 232   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [23]:
print(X_train[4])

[ 14 356  83 126   5 742  59   5 357  96  41 127 234   3  25 161   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


# Model Creation

In [0]:
#from keras.models import Sequential
#from keras import layers
import tensorflow as tf

m = tf.keras.models
layers = tf.keras.layers

In [0]:
model = m.Sequential()

In [65]:
# vocab size is 1750 
# input_length is size of review text after tokenization and pad sequance
embedding_dim = 80

model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 80)           139760    
_________________________________________________________________
flatten_5 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                80010     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 219,781
Trainable params: 219,781
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [67]:
history = model.fit(X_train, y_train,
                    epochs=18,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)



Epoch 1/18
75/75 [==============================] - 0s 5ms/step - loss: 0.6959 - accuracy: 0.5013 - val_loss: 0.6906 - val_accuracy: 0.5200
Epoch 2/18
75/75 [==============================] - 0s 4ms/step - loss: 0.6748 - accuracy: 0.6053 - val_loss: 0.6973 - val_accuracy: 0.4840
Epoch 3/18
75/75 [==============================] - 0s 4ms/step - loss: 0.5181 - accuracy: 0.8453 - val_loss: 0.5964 - val_accuracy: 0.6640
Epoch 4/18
75/75 [==============================] - 0s 4ms/step - loss: 0.2524 - accuracy: 0.9547 - val_loss: 0.5613 - val_accuracy: 0.7200
Epoch 5/18
75/75 [==============================] - 0s 4ms/step - loss: 0.1054 - accuracy: 0.9853 - val_loss: 0.5347 - val_accuracy: 0.7320
Epoch 6/18
75/75 [==============================] - 0s 4ms/step - loss: 0.0508 - accuracy: 0.9960 - val_loss: 0.5490 - val_accuracy: 0.7600
Epoch 7/18
75/75 [==============================] - 0s 4ms/step - loss: 0.0286 - accuracy: 0.9973 - val_loss: 0.6029 - val_accuracy: 0.7320
Epoch 8/18
75/75 [==

In [68]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))


Training Accuracy: 1.0000


In [69]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))


Testing Accuracy: 0.7800


# Let's do the prediction

In [76]:
import numpy as np
phrase = "food was stale"
#phrase = "bad service"

# Use same as tokenzier object which was used for training data
tokens = tokenizer.texts_to_sequences([phrase])

# same as for training
pad_tokens = pad_sequences(tokens, padding='post', maxlen=maxlen)

print(tokens)
print(pad_tokens)

[[10, 3, 611]]
[[ 10   3 611   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]]


In [0]:
val = model.predict_classes(pad_tokens)   

In [78]:
print(val[0][0])

0


In [0]:
def predictSentiments ( indexvalue):
    
    reviewSentiment = ''
    
    if (val[0][0] == 0):
        reviewSentiment = 'Customer is gone forever,'
    else:
        reviewSentiment = 'you got back your customer'

    return reviewSentiment;

In [74]:
print(predictSentiments(val[0][0]))

you got back your customer


# Save the model to re-use later

In [0]:

import pickle

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

# Deletes the existing model
#del model  


# saving tokenizer 
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



# Load Model and tokenizer

In [81]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer_saved = pickle.load(handle)

# Returns a compiled model identical to the previous one
model_saved = m.load_model('my_model.h5')

In [82]:
#review_sen = "good food ,will come again"
review_sen = "bad service"

tokens_sen = tokenizer_saved.texts_to_sequences([review_sen])
pad_tokens_sen = pad_sequences(tokens_sen, padding='post', maxlen=maxlen)

print(tokens_sen)
print(pad_tokens_sen)

[[90, 19]]
[[90 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]]


In [83]:
val = model_saved.predict_classes(pad_tokens_sen)
print(predictSentiments(val[0][0]))

Customer is gone forever,


# Another model

In [0]:
model2 = Sequential()

model2.add(layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=maxlen))

model2.add(layers.GlobalMaxPool1D())

model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.summary()

In [0]:
history2 = model2.fit(X_train, y_train,
                    epochs=20,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

In [0]:
loss, accuracy = model2.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))



In [0]:
loss, accuracy = model2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))



# Using Pre-Trained GloVe vector

In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    
    vocab_size = len(word_index) + 1 
    # Adding again 1 because of reserved 0 index
    
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as file:
        for line in file:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                print("{} {} ".format(word,idx))
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

In [0]:
embedding_dim = 50

filePath = 'GloVe_PreTrained/glove.6B.50d.txt'

embedding_matrix = create_embedding_matrix(filePath,
                                           tokenizer.word_index, 
                                           embedding_dim)

In [0]:
print(embedding_matrix[0:2])

In [0]:
model3 = Sequential()

model3.add(layers.Embedding(vocab_size, 
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)) # Make it False
#model3.add(layers.Conv1D(128, 5, activation='relu'))
model3.add(layers.GlobalMaxPool1D())

model3.add(layers.Dense(10, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model3.summary()

In [0]:
history3 = model3.fit(X_train, y_train,
                    epochs=20,verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

In [0]:
loss, accuracy = model3.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

In [0]:
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
